## Exercício: LoRA

- Exercício didático para entender a técnica de fazer ajuste fino em modelos grandes usando poucos recursos
- Aplicar no pré exercício de análise de sentimento ou no segundo exercício, e modelo de linguagem, com vocabulário de 3000 palavras, embedding size e 2 camadas, treinados da forma usual (medir tempo de treinamento/época)
- Modificar o seu modelo para adotar a técnica do LoRA no embedding e nas 2 camadas, e fazer o ajuste-fino, isto é, continuar o treinamento anterior, lembrando que as matrizes originais ficarão congeladas e o ajuste dos pesos serão apenas aplicados nas matrizes do LoRA. Medir o tempo de treinamento/época.
- Por último, substituir o modelo original, com os novos pesos calculados pelo W + LoRA.

In [1]:
import os
import sys
import random
import time
import re
from collections import Counter
from sklearn.model_selection import train_test_split

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
# Global variables

# Vocabulary
vocab_size = 3000
context_size = 9
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 32
epochs = 10
lr = 0.001

# Model
embedding_dim = 64
hidden_dim = 200

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Faz download e carrega o dataset

In [4]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

### Limpeza do texto principal

In [5]:
text_1 = open("67724.txt.utf-8","r").read()
text_2 = open("67725.txt.utf-8","r").read()

def clean_text(text):
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"
    text_start = text.find(start_marker)
    text_end = text.find(end_marker)

    text_content= text[text_start:text_end].replace('\r','')
    paragraphs = []
    for paragraph in text_content.split("\n\n"):
        paragraph = paragraph.replace('\n', ' ').strip()
        # Validation of length and index lines
        if (len(paragraph) > 10 and '....' not in paragraph):
            paragraphs.append(paragraph)
    return paragraphs

cleaned_paragraphs = clean_text(text_1)+clean_text(text_2)
print(f'Number of paragraphs: {len(cleaned_paragraphs)}')

Number of paragraphs: 4596


## Análise do dataset

In [6]:
# Conta as palavras no dataset
def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

11875

## Criando um vocabulário

In [7]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [8]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: [',', '.', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 3000


#### Codificação / Decodificação das sentenças

In [84]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

def decode_sentence(encoded_sentence, vocab):
    words = []
    for index in encoded_sentence:
        word = next((word for word, code in vocab.items() if code == index), "<UNK>")
        words.append(word)

    return words

seq = cleaned_paragraphs[20]
spc = ' '
encoded = encode_sentence(seq, vocab)
decoded = decode_sentence(encoded, vocab)

print(f'Original Seq: {seq}')
print(f'Encoded: {encoded}')
print(f'Decoded: {decoded}')
print(f'Reconstructed Seq: {spc.join(decoded)}')

Original Seq: Ahi, o _Paquequer_ lança-se rapido sobre o seu leito, e atravessa as florestas como o tapir, espumando, deixando o pello esparso pelas pontas de rochedo, e enchendo a solidão com o estampido de sua carreira. De repente, falta-lhe o espaço, foge-lhe a terra; o soberbo rio recúa um momento para concentrar as suas forças e precipita-se de um só arremesso, como o tigre sobre a presa.
Encoded: [235, 1, 5, 723, 0, 8, 762, 38, 5, 19, 324, 1, 7, 0, 23, 634, 28, 5, 2447, 1, 0, 1, 763, 5, 1776, 0, 269, 1065, 6, 486, 1, 7, 2448, 3, 687, 16, 5, 2449, 6, 17, 1777, 2, 6, 240, 1, 522, 29, 5, 612, 1, 0, 29, 3, 128, 9, 5, 1577, 99, 0, 10, 72, 18, 0, 23, 87, 591, 7, 2450, 8, 6, 10, 74, 0, 1, 28, 5, 592, 38, 3, 979, 2]
Decoded: ['ahi', ',', 'o', '_paquequer_', '<UNK>', 'se', 'rapido', 'sobre', 'o', 'seu', 'leito', ',', 'e', '<UNK>', 'as', 'florestas', 'como', 'o', 'tapir', ',', '<UNK>', ',', 'deixando', 'o', 'pello', '<UNK>', 'pelas', 'pontas', 'de', 'rochedo', ',', 'e', 'enchendo', 'a', 's

## Classe do dataset

In [10]:
# Dataset class
class BagOfWordsDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [11]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = BagOfWordsDataset(train_data, vocab, context_size)
val_dataset = BagOfWordsDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3676
Validation samples: 920

Training dataset samples: 24360
Validation dataset samples: 5851


In [12]:
tst_loader = DataLoader(train_dataset, batch_size = 1, shuffle=True)
sample = next(iter(tst_loader))
print(sample)

[tensor([[394,  21,   4,   8, 506,   5, 315,   6,  34]]), tensor([2])]


In [13]:
# Train/val loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Model

In [14]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h, bias=True)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1, bias=True)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        # Relu 
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)

        return out

In [15]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

#### Teste básico do modelo

In [16]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

output = model(input)
pred = output.argmax(dim=1)

print(pred)
print(target)

torch.Size([32, 9])
torch.Size([32])
tensor([1435, 2701, 1221,  633, 2053, 2462, 2327, 1546,  817,  647,  633, 1184,
         426, 2421, 1984,  633,  943,   73, 1059, 1184, 2239, 2715, 2882, 1275,
        2761, 1546, 2157,  223,  223,  633,  830, 1221])
tensor([  34,    6,    8,   41,  803,    2,  357,   17,    1,    4,   40,   95,
          10,    1,    1,    3,   10,   41,   39, 1190,  209,   90,  372,   70,
         625,   18,  718,    1, 1370, 1443,    1,    7])


## Training

### Funções de Treinamento e Avaliação do Modelo

#### Função para Contagem de Parâmetros do Modelo

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 910,665 parâmetros.


#### Função para Avaliação Inicial do Modelo

In [18]:
def init_eval():
    # Initial Perplexity and Loss
    # Before training
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()

    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

#### Função para Treinamento do Modelo

In [19]:
def train():
      # Training Loop
      model.train()
      for epoch in range(epochs):

            epoch_start = time.time()
            # Metrics
            epoch_loss = 0
            epoch_correct = 0
            epoch_samples = 0

            for inputs, targets in train_loader:
                  inputs = inputs.to(device)  # Move input data to the device
                  targets = targets.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)

                  # Backward pass and optimization
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

                  # Loss
                  epoch_loss += loss.item()

                  # Predicted
                  predicted = outputs.argmax(dim=1)
                  epoch_correct += (predicted == targets).sum().item()
                  epoch_samples += targets.size(0)

            # Calculate average loss and accuracy for epoch
            avg_loss = epoch_loss / len(train_loader)
            acc = epoch_correct / epoch_samples

            # Perplexity
            perp = torch.exp(torch.tensor(avg_loss))

            epoch_end = time.time()
            epoch_time = epoch_end - epoch_start
            # Print epoch statistics
            print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


#### Função para Avaliação na Base de Validação

In [20]:
def eval():
    model.eval()

    loss_sum = 0
    total_sum = 0
    correct_sum = 0
    eval_round = 0

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)      
            loss_sum += loss

            # Get the predicted labels
            predicted = outputs.argmax(dim=1)

            total_sum += targets.size(0)
            correct_sum += (predicted == targets).sum().item()
            eval_round += 1

    # Calculate accuracy
    acc = 100 * correct_sum / total_sum

    # Calculate average perplexity
    average_loss = loss_sum / len(val_loader)
    average_perplexity = torch.exp(average_loss)

    print(f'Test Accuracy: {acc:.2f}%')
    print(f'Average Loss: {average_loss:.2f}')
    print(f'Average Perplexity: {average_perplexity:.2f}')

### Treinamento e Avaliação

In [21]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)
print(model)
count_parameters(model)

BengioModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=576, out_features=200, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=200, out_features=3001, bias=True)
)


910665

In [22]:
print("Initial Evaluation")
print()
init_eval()
print()
print("Training the Model")
print()
train()
print()
print("Evaluation on the Validation Dataset")
eval()

Initial Evaluation

Initial Loss: 8.0423
Initial Perplexity: 3109.8086

Training the Model

Epoch [1/10], Time:2.76, Loss: 7.9736, Accuracy: 0.00%, Perplexity: 2903.2432
Epoch [2/10], Time:2.64, Loss: 7.8292, Accuracy: 0.03%, Perplexity: 2512.8303
Epoch [3/10], Time:2.72, Loss: 7.6685, Accuracy: 0.05%, Perplexity: 2139.8926
Epoch [4/10], Time:2.82, Loss: 7.4744, Accuracy: 0.06%, Perplexity: 1762.2626
Epoch [5/10], Time:2.71, Loss: 7.2319, Accuracy: 0.06%, Perplexity: 1382.8102
Epoch [6/10], Time:2.67, Loss: 6.9625, Accuracy: 0.06%, Perplexity: 1056.2560
Epoch [7/10], Time:2.67, Loss: 6.7537, Accuracy: 0.06%, Perplexity: 857.2641
Epoch [8/10], Time:2.41, Loss: 6.6226, Accuracy: 0.07%, Perplexity: 751.9162
Epoch [9/10], Time:2.57, Loss: 6.5371, Accuracy: 0.08%, Perplexity: 690.3011
Epoch [10/10], Time:2.51, Loss: 6.4710, Accuracy: 0.08%, Perplexity: 646.1207

Evaluation on the Validation Dataset
Test Accuracy: 7.64%
Average Loss: 6.49
Average Perplexity: 657.21


## Geração de Sentenças

In [128]:
# Get a random sentence of context size

seq = "O indio atravessou a sala, e collocando-se"
inputs = encode_sentence(seq, vocab)
inputs = inputs[len(inputs)-context_size:len(inputs)]
inputs = torch.tensor([inputs])

new_tokens = 10
with torch.no_grad():
    context = torch.tensor(inputs, dtype=torch.long).squeeze().to(device)
    
    for _ in range(new_tokens):

        output = model(torch.tensor(inputs).to(device))
        probs = F.softmax(output, dim=1)
        next_token = torch.multinomial(probs, num_samples=1).squeeze()       
        context = torch.cat([context, next_token.reshape(1)], dim=0)
        
' '.join(decode_sentence(context.tolist(), vocab))

C:\Users\fabio.grassiotto\AppData\Local\Temp\ipykernel_12660\2865815520.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context = torch.tensor(inputs, dtype=torch.long).squeeze().to(device)
C:\Users\fabio.grassiotto\AppData\Local\Temp\ipykernel_12660\2865815520.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs).to(device))


'o indio atravessou a sala , e <UNK> se alpendrada talhe que e poesia , . e perolas chegar'